In [ ]:
import os, json, numpy as np
from PIL import Image
import tensorflow as tf

# For Colab file upload
try:
    from google.colab import files
except:
    files = None


In [ ]:
IMG_SIZE = 224

model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=True,
    weights='imagenet'
)


14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def load_image_for_model(path):
    img = Image.open(path)

    if img.mode == "P":
        img = img.convert("RGBA").convert("RGB")
    else:
        img = img.convert("RGB")

    img = img.resize((IMG_SIZE, IMG_SIZE))
    arr = np.array(img).astype("float32")
    arr = tf.keras.applications.mobilenet_v2.preprocess_input(arr)

    return np.expand_dims(arr, 0)


In [ ]:
def predict_dish(image_path, model):
    x = load_image_for_model(image_path)
    preds = model.predict(x)
    decoded = tf.keras.applications.mobilenet_v2.decode_predictions(preds, top=1)[0]
    _, label, conf = decoded[0]
    return label, float(conf)


In [ ]:
def generate_recipe_local(dish_name, top_ingredients=None):
    quick_map = {
        "espresso": {
            "title": "Espresso",
            "ingredients": [
                {"name": "ground coffee", "qty": "18-20 g"},
                {"name": "water", "qty": "30-40 ml"}
            ],
            "steps": [
                "Tamp the coffee into the portafilter.",
                "Brew for 25–30 seconds.",
                "Serve immediately."
            ]
        },
        "pizza": {
            "title": "Pizza",
            "ingredients": [
                {"name": "pizza base", "qty": "1"},
                {"name": "tomato sauce", "qty": "2 tbsp"},
                {"name": "mozzarella", "qty": "50 g"}
            ],
            "steps": [
                "Spread sauce on the base.",
                "Add mozzarella.",
                "Bake at 220°C for 8–10 minutes."
            ]
        },
        "beet_salad": {
            "title": "Beet Salad",
            "ingredients": [
                {"name": "boiled beetroot", "qty": "1 cup"},
                {"name": "olive oil", "qty": "1 tbsp"},
                {"name": "lemon juice", "qty": "1 tsp"}
            ],
            "steps": [
                "Chop beetroot.",
                "Mix with oil and lemon.",
                "Serve chilled."
            ]
        }
    }

    key = dish_name.lower().replace(" ", "_")

    if key in quick_map:
        return quick_map[key]

    # Generic fallback
    ingredients = [{"name": ing, "qty": "to taste"} for ing in (top_ingredients or ["salt", "oil"])]

    return {
        "title": dish_name,
        "ingredients": ingredients,
        "steps": [
            f"Prepare main ingredients for {dish_name}.",
            "Cook or assemble appropriately.",
            "Serve fresh."
        ]
    }


In [ ]:
def generate_recipe_openai(dish_name, top_ingredients=None):
    try:
        import openai
    except:
        return None, "openai not installed"

    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        return None, "OPENAI_API_KEY not set"

    openai.api_key = api_key

    prompt = f"""
Return only a JSON object with: title, ingredients (name & qty), steps list.
Dish: {dish_name}
Keep steps short.
"""

    try:
        resp = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=300
        )
        text = resp["choices"][0]["message"]["content"].strip()

        json_text = text[text.find("{"):text.rfind("}")+1]
        data = json.loads(json_text)
        return data, None
    except Exception as e:
        return None, str(e)


In [ ]:
SAMPLE_RECIPES = {}

def predict_and_generate(image_path, use_openai=True):
    label, conf = predict_dish(image_path, model)
    print(f"Predicted: {label}  (conf={conf:.3f})")

    key = label.lower().replace(" ", "_")

    # Cache check
    if key in SAMPLE_RECIPES:
        return label, conf, SAMPLE_RECIPES[key]

    # Try LLM
    if use_openai:
        recipe_data, error = generate_recipe_openai(label)
        if recipe_data:
            SAMPLE_RECIPES[key] = recipe_data
            return label, conf, recipe_data
        else:
            print("LLM error:", error)

    # Fallback
    recipe_data = generate_recipe_local(label)
    SAMPLE_RECIPES[key] = recipe_data
    return label, conf, recipe_data


In [ ]:
def choose_image():
    uploaded = files.upload()
    filename = next(iter(uploaded))
    return "/content/" + filename

# Upload image
image_path = choose_image()

# Predict & Generate
label, conf, recipe = predict_and_generate(image_path, use_openai=True)

# Display recipe
print("\nIngredients:")
for ing in recipe["ingredients"]:
    print("-", ing)

print("\nSteps:")
for i, step in enumerate(recipe["steps"], 1):
    print(f"{i}. {step}")


Saving coffee.jpg to coffee.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Predicted: espresso  (conf=0.939)
LLM error: OPENAI_API_KEY not set

Ingredients:
- {'name': 'ground coffee', 'qty': '18-20 g'}
- {'name': 'water', 'qty': '30-40 ml'}

Steps:
1. Tamp the coffee into the portafilter.
2. Brew for 25–30 seconds.
3. Serve immediately.
